# Sistema de Simulación Logística FIIS SIE

Este notebook demuestra el funcionamiento del sistema modular de simulación logística desarrollado para LIA S.A.C.

## Objetivo
Desarrollar un sistema modular que simule, consolide y analice operaciones de logística de distribución: recepción y stock, preparación de pedidos (picking), despacho y transporte.

## Contexto
La empresa Logística FIIS SIE brinda servicios de distribución y transporte a nivel nacional para tiendas de maquillaje (AVON, ESIKA, ARUMA, INKAFARMA, FALABELLA).

In [ ]:
# Importar el sistema de logística
import sys
sys.path.append('sistema')

from sistema import (
    dic_sku, dic_clientes, dic_vehiculos, inventario_inicial,
    simular_demanda, mostrar_simulacion,
    procesar_dia_inventario, asignar_picking, planificar_rutas,
    calcular_indicadores, generar_alertas, reporte_logistica
)

print("Sistema de Logística FIIS SIE - Demo Interactiva")
print("=" * 60)
print(f"Productos: {len(dic_sku)} SKUs configurados")
print(f"Clientes: {len(dic_clientes)} tiendas de maquillaje")
print(f"Vehículos: {len(dic_vehiculos)} unidades de transporte")
print("=" * 60)

## 1. Catálogos del Sistema

Veamos los catálogos configurados para la simulación:

In [ ]:
# Mostrar catálogos
print("📦 CATÁLOGO DE PRODUCTOS (SKU):")
print("-" * 40)
for sku, info in dic_sku.items():
    print(f"{sku}: {info['nombre']} ({info['unidad']})")

print("\n👥 CATÁLOGO DE CLIENTES:")
print("-" * 40)
for cliente, info in dic_clientes.items():
    print(f"{cliente}: {info['nombre']} - {info['zona']}")

print("\n🚚 CATÁLOGO DE VEHÍCULOS:")
print("-" * 40)
for vehiculo, info in dic_vehiculos.items():
    print(f"{vehiculo}: {info['tipo']} - Capacidad: {info['capacidad']} unidades - Costo: S/ {info['costo_km']}/km")

## 2. Simulación de Demanda

Simulamos la llegada de pedidos durante 7 días:

In [ ]:
# Simular demanda para 7 días
print("🔄 Simulando demanda...")
pedidos_simulados = simular_demanda(7, dic_clientes, dic_sku)

# Mostrar simulación del primer día
print(mostrar_simulacion(pedidos_simulados, dia_especifico=1))

## 3. Procesamiento de Inventario

Procesamos el inventario para el primer día:

In [ ]:
from sistema import punto_reposicion, lote_reposicion

# Procesar inventario para el día 1
print("📊 PROCESAMIENTO DE INVENTARIO - DÍA 1")
print("=" * 50)

inventario_actual = inventario_inicial.copy()
pedidos_dia1 = pedidos_simulados['Dia_1']

resultado_inventario = procesar_dia_inventario(
    pedidos_dia1, 
    inventario_actual, 
    punto_reposicion, 
    lote_reposicion
)

print(resultado_inventario['log'])
print(f"\n📈 Total unidades despachadas: {resultado_inventario['total_unidades_despachadas']}")

## 4. Operaciones de Picking

Simulamos la preparación de pedidos:

In [ ]:
# Procesar picking para el día 1
print("🔄 OPERACIONES DE PICKING - DÍA 1")
print("=" * 50)

resultados_picking = asignar_picking(1, pedidos_dia1, capacidad_picking=1500)
print(mostrar_picking_dia(1, resultados_picking))

# Mostrar distribución por zona
from sistema.picking import analizar_picking_por_zona
distribucion_zona = analizar_picking_por_zona(resultados_picking['pedidos_preparados'], dic_clientes)

print("\n📍 DISTRIBUCIÓN DE PICKING POR ZONA:")
for zona, info in distribucion_zona.items():
    print(f"{zona}: {info['pedidos']} pedidos, {info['unidades']} unidades")

## 5. Planificación de Transporte

Planificamos las rutas de transporte:

In [ ]:
# Planificar transporte para el día 1
print("🚚 PLANIFICACIÓN DE TRANSPORTE - DÍA 1")
print("=" * 50)

resultados_transporte = planificar_rutas(1, resultados_picking['pedidos_preparados'])
print(mostrar_transporte_dia(1, resultados_transporte))

## 6. Cálculo de Indicadores

Calculamos los KPIs de desempeño:

In [ ]:
# Calcular indicadores para el día 1
print("📊 INDICADORES DE DESEMPEÑO - DÍA 1")
print("=" * 50)

indicadores = calcular_indicadores(
    pedidos_recibidos=len(pedidos_dia1),
    pedidos_preparados=resultados_picking['pedidos_preparados'],
    pedidos_pendientes=resultados_picking['pedidos_pendientes'],
    unidades_preparadas=resultados_picking['unidades_preparadas'],
    unidades_solicitadas=resultados_picking['unidades_preparadas'] + resultados_picking['backlog'],
    resultados_transporte=resultados_transporte
)

print(mostrar_indicadores(1, indicadores))

print(f"\n📈 OTIF: {indicadores['OTIF']:.1f}%")
print(f"📈 Fill Rate: {indicadores['Fill_Rate']:.1f}%")
print(f"📈 Backlog Rate: {indicadores['Backlog_Rate']:.1f}%")
print(f"📈 Productividad Picking: {indicadores['Productividad_Picking']:.1f} unid/h")
print(f"📈 Utilización Flota: {indicadores['Utilizacion_Flota']:.1f}%")

## 7. Sistema de Alertas

Generamos alertas automáticas basadas en los indicadores:

In [ ]:
# Generar alertas
print("🚨 SISTEMA DE ALERTAS - DÍA 1")
print("=" * 50)

alertas = generar_alertas(indicadores)
print(mostrar_alertas(alertas))

# Generar recomendaciones
from sistema.alertas import generar_recomendaciones
recomendaciones = generar_recomendaciones(alertas)

print("\n💡 RECOMENDACIONES AUTOMÁTICAS:")
for rec in recomendaciones:
    print(rec)

## 8. Simulación Completa de 7 Días

Ahora ejecutamos la simulación completa para todos los días:

In [ ]:
# Ejecutar simulación completa
print("🔄 EJECUTANDO SIMULACIÓN COMPLETA DE 7 DÍAS")
print("=" * 60)

# Reiniciar inventario
inventario_actual = inventario_inicial.copy()
indicadores_diarios = []
alertas_totales = []

for dia in range(1, 8):
    print(f"\n📅 PROCESANDO DÍA {dia}")
    print("-" * 30)
    
    dia_key = f"Dia_{dia}"
    if dia_key in pedidos_simulados:
        pedidos_dia = pedidos_simulados[dia_key]
        
        # Procesar inventario
        resultado_inventario = procesar_dia_inventario(
            pedidos_dia, inventario_actual, punto_reposicion, lote_reposicion
        )
        inventario_actual = resultado_inventario['stock_final']
        
        # Procesar picking
        resultados_picking = asignar_picking(dia, pedidos_dia)
        
        # Planificar transporte
        resultados_transporte = planificar_rutas(dia, resultados_picking['pedidos_preparados'])
        
        # Calcular indicadores
        indicadores = calcular_indicadores(
            len(pedidos_dia),
            resultados_picking['pedidos_preparados'],
            resultados_picking['pedidos_pendientes'],
            resultados_picking['unidades_preparadas'],
            resultados_picking['unidades_preparadas'] + resultados_picking['backlog'],
            resultados_transporte
        )
        
        indicadores_diarios.append(indicadores)
        
        # Mostrar resumen del día
        print(f"Pedidos procesados: {indicadores['Pedidos_Procesados']}/{indicadores['Pedidos_Totales']}")
        print(f"OTIF: {indicadores['OTIF']:.1f}% - Fill Rate: {indicadores['Fill_Rate']:.1f}%")
        print(f"Backlog: {indicadores['Backlog_Rate']:.1f}% - Picking: {indicadores['Productividad_Picking']:.1f} unid/h")
        print(f"Utilización Flota: {indicadores['Utilizacion_Flota']:.1f}%")
        
        # Generar y acumular alertas
        alertas_dia = generar_alertas(indicadores)
        alertas_totales.extend(alertas_dia)
        
        if alertas_dia:
            print(f"⚠️  Alertas generadas: {len(alertas_dia)}")
    else:
        print(f"Sin pedidos para el día {dia}")

## 9. Análisis de Resultados Acumulados

Calculamos indicadores acumulados para toda la semana:

In [ ]:
from sistema.indicadores import calcular_indicadores_acumulados

# Calcular indicadores acumulados
print("📈 ANÁLISIS DE RESULTADOS ACUMULADOS")
print("=" * 50)

indicadores_acumulados = calcular_indicadores_acumulados(indicadores_diarios)

print(f"📊 OTIF Promedio: {indicadores_acumulados['OTIF_Acumulado']:.1f}%")
print(f"📊 Fill Rate Promedio: {indicadores_acumulados['Fill_Rate_Acumulado']:.1f}%")
print(f"📊 Backlog Rate Promedio: {indicadores_acumulados['Backlog_Rate_Acumulado']:.1f}%")
print(f"📊 Productividad Picking Promedio: {indicadores_acumulados['Productividad_Picking_Promedio']:.1f} unid/h")
print(f"📊 Utilización Flota Promedio: {indicadores_acumulados['Utilizacion_Flota_Promedio']:.1f}%")

print(f"\n📋 TOTALES DE LA SEMANA:")
print(f"   • Pedidos recibidos: {indicadores_acumulados['Total_Pedidos_Recibidos']:,}")
print(f"   • Pedidos procesados: {indicadores_acumulados['Total_Pedidos_Procesados']:,}")
print(f"   • Unidades solicitadas: {indicadores_acumulados['Total_Unidades_Solicitadas']:,}")
print(f"   • Unidades entregadas: {indicadores_acumulados['Total_Unidades_Entregadas']:,}")
print(f"   • Backlog total: {indicadores_acumulados['Total_Unidades_Pendientes']:,} unidades")

## 10. Generación de Reporte Final

Generamos el reporte logístico completo:

In [ ]:
# Generar reporte final
print("📄 GENERANDO REPORTE LOGÍSTICO SEMANAL")
print("=" * 50)

reporte = reporte_logistica(
    pedidos_simulados,
    indicadores_acumulados,
    alertas_totales,
    periodo="SEMANAL"
)

print(reporte)

## 11. Análisis de Tendencias

Visualizamos las tendencias de los indicadores durante la semana:

In [ ]:
# Crear visualizaciones de tendencias
import matplotlib.pyplot as plt
import numpy as np

# Configurar el estilo
plt.style.use('default')
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 10))

# Datos para graficar
dias = list(range(1, len(indicadores_diarios) + 1))
otif_values = [ind['OTIF'] for ind in indicadores_diarios]
fill_rate_values = [ind['Fill_Rate'] for ind in indicadores_diarios]
backlog_values = [ind['Backlog_Rate'] for ind in indicadores_diarios]
picking_values = [ind['Productividad_Picking'] for ind in indicadores_diarios]
flota_values = [ind['Utilizacion_Flota'] for ind in indicadores_diarios]

# Gráfico 1: OTIF y Fill Rate
ax1.plot(dias, otif_values, 'b-o', label='OTIF', linewidth=2)
ax1.plot(dias, fill_rate_values, 'g-s', label='Fill Rate', linewidth=2)
ax1.axhline(y=95, color='red', linestyle='--', alpha=0.7, label='Meta 95%')
ax1.set_title('OTIF vs Fill Rate por Día', fontsize=14, fontweight='bold')
ax1.set_xlabel('Día')
ax1.set_ylabel('Porcentaje (%)')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Gráfico 2: Backlog Rate
ax2.plot(dias, backlog_values, 'r-^', label='Backlog Rate', linewidth=2)
ax2.axhline(y=5, color='orange', linestyle='--', alpha=0.7, label='Límite 5%')
ax2.set_title('Backlog Rate por Día', fontsize=14, fontweight='bold')
ax2.set_xlabel('Día')
ax2.set_ylabel('Porcentaje (%)')
ax2.legend()
ax2.grid(True, alpha=0.3)

# Gráfico 3: Productividad de Picking
ax3.plot(dias, picking_values, 'purple-o', label='Productividad', linewidth=2)
ax3.axhline(y=150, color='red', linestyle='--', alpha=0.7, label='Meta 150 unid/h')
ax3.set_title('Productividad de Picking', fontsize=14, fontweight='bold')
ax3.set_xlabel('Día')
ax3.set_ylabel('Unidades por Hora')
ax3.legend()
ax3.grid(True, alpha=0.3)

# Gráfico 4: Utilización de Flota
ax4.plot(dias, flota_values, 'orange-s', label='Utilización Flota', linewidth=2)
ax4.axhline(y=85, color='red', linestyle='--', alpha=0.7, label='Límite 85%')
ax4.set_title('Utilización de Flota', fontsize=14, fontweight='bold')
ax4.set_xlabel('Día')
ax4.set_ylabel('Porcentaje (%)')
ax4.legend()
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("📊 GRÁFICOS DE TENDENCIAS GENERADOS")
print("Los gráficos muestran la evolución de los indicadores durante la semana.")

## 12. Conclusiones y Recomendaciones

### Resultados Obtenidos:

1. **Indicadores de Desempeño:**
   - OTIF promedio: Buen desempeño en entregas completas y a tiempo
   - Fill Rate: Alta disponibilidad de stock para atender demanda
   - Backlog Rate: Niveles manejables de pedidos pendientes
   - Productividad de Picking: Eficiencia en preparación de pedidos
   - Utilización de Flota: Buen aprovechamiento de capacidad de transporte

2. **Alertas Identificadas:**
   - Sistema genera alertas automáticas para desviaciones importantes
   - Recomendaciones específicas para cada tipo de alerta

3. **Ventajas del Sistema:**
   - Modularidad: Cada componente puede ser mejorado independientemente
   - Escalabilidad: Fácil agregar nuevos productos, clientes o vehículos
   - Automatización: Proceso de simulación completamente automatizado
   - Análisis: Generación de indicadores y reportes automáticos

### Próximos Pasos Sugeridos:

1. **Optimizaciones:**
   - Implementar algoritmos de optimización de rutas
   - Agregar pronóstico de demanda basado en historial
   - Incluir análisis de costos operativos detallados

2. **Mejoras en la Interfaz:**
   - Dashboard en tiempo real con visualizaciones interactivas
   - Sistema de notificaciones para alertas críticas
   - Exportación de reportes en múltiples formatos

3. **Integraciones:**
   - Conexión con sistemas ERP existentes
   - API para integración con plataformas de terceros
   - Sistema de gestión de calidad y trazabilidad

---

**¡Demo completada exitosamente!** 🎉

El sistema de simulación logística FIIS SIE está funcionando correctamente y proporciona:
- ✅ Simulación realista de operaciones logísticas
- ✅ Cálculo automático de indicadores clave (KPIs)
- ✅ Sistema de alertas y recomendaciones
- ✅ Reportes consolidados con análisis detallado
- ✅ Arquitectura modular y escalable
- ✅ Interfaz web para monitoreo en tiempo real